# 청결도 분류기 평가

## 필요한 라이브러리 한번에 다운로드

In [ ]:
!pip install -r ../requirements.txt

In [ ]:
!pip install -e ../mmdetection

In [ ]:
!mim install "mmengine>=0.7.0"
!mim install "mmcv>=2.0.0rc4"

In [ ]:
import sys
sys.path.append('../') 
sys.path.append('../mmdetection')

## 파일 이름 변경하는 코드 

In [ ]:
import os

# data 폴더 경로
data_dir = 'data'
output_dir = 'clean_rating_data'

# output 디렉토리 생성
os.makedirs(output_dir, exist_ok=True)

# data 폴더 내의 모든 하위 폴더에 대해 반복
for label_folder in os.listdir(data_dir):
    label_folder_path = os.path.join(data_dir, label_folder)

    # 폴더인 경우에만 처리
    if os.path.isdir(label_folder_path):

        # 폴더 내의 각 파일에 대해 반복
        for filename in os.listdir(label_folder_path):
            file_path = os.path.join(label_folder_path, filename)

            # 파일이 실제로 파일이고 이미지 파일인 경우
            if os.path.isfile(file_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
                # 폴더 이름을 라벨로 사용하여 파일 이름 변경
                new_filename = f'{label_folder}_{filename}'
                new_file_path = os.path.join(output_dir, new_filename)
                # 파일 이동
                os.rename(file_path, new_file_path)

## 디렉토리에 있는 파일 가져오기 

In [ ]:
import os

def get_image_paths(data_dir):
    image_list = []

    # data 폴더 내의 모든 파일에 대해 반복
    for filename in os.listdir(data_dir):
        file_path = os.path.join(data_dir, filename)

        # 파일이 실제로 파일이고 이미지 파일인 경우
        if os.path.isfile(file_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
            image_list.append(file_path)

    return image_list

In [ ]:
data_dir = './clean_rating_data'
images = get_image_paths(data_dir)

# 15장만 테스트
images = images[:15]

## 청결도 예측하기 

In [ ]:
# true 라벨값 불러오기 
import os

def label_to_number(label):
    label_mapping = {'pending': 0, 'clean': 1, 'normal': 2, 'dirty': 3}
    return label_mapping.get(label, -1)

def process_files_with_labels(image_list):
    y_true = []
    
    # 파일마다 반복
    for file_path in image_list:
        
        # 파일 이름을 공백을 기준으로 분리하여 라벨 추출
        filename = file_path.split('\\')[1].lower()
        label = filename.split('_')[0].lower()
        
        # 라벨을 숫자로 변환
        label_number = label_to_number(label)
        y_true.append(label_number)
        
    return y_true
    
# 함수 호출
y_true = process_files_with_labels(images)

In [ ]:
config_file = '../mmdetection/configs/dino/dino-4scale_r50_8xb2-60e_coco_all_v2.py'
checkpoint_file = './model/weights/mmdetection_dino_weights.pth'
yolov8_checkpoint_file = './model/weights/yolov8_weights.pt'

In [ ]:
# # 파일 열기 (읽기 모드)
# with open('y_pred.txt', 'r') as file:
#     # 파일의 각 줄을 읽어와 리스트에 저장
#     y_preds = [int(line.strip()) for line in file]

In [ ]:
y_preds=[]

In [ ]:
# mmdetection , yolo , ensemble
from inference.predict_list import mmdetection_list_predict
from inference.predict_list import yolov8_list_predict
from inference.predict_list import ensemble_list_predict
from inference.clean_rating import determine_clean_class

for idx in range(0, len(images), 10):
    end_idx = min(idx + 10, len(images))  # Ensure not to exceed the length of the images list
    mmdetection_predictions = mmdetection_list_predict(images[idx:end_idx], config_file, checkpoint_file)
    yolov8_predictions = yolov8_list_predict(images[idx:end_idx], yolov8_checkpoint_file)
    wbf_predictions = ensemble_list_predict(mmdetection_predictions, yolov8_predictions)
    y_pred = []     
    for wbf_prediction in wbf_predictions:
        y_pred.append(determine_clean_class(wbf_prediction))
    y_preds.extend(y_pred)
    print(f"---------------Iteration {idx}: Processed images {end_idx} / {len(images)}---------------")

In [ ]:
# 중간에 멈췄을 때 파일 저장하고 불러오는 코드 
# 파일 열기 (쓰기 모드)
# with open('y_preds.txt', 'w') as file:
#     # 리스트의 각 항목을 파일에 쓰기
#     for item in y_preds:
#         file.write(str(item) + '\n')

# # 파일 열기 (읽기 모드)
# with open('y_pred.txt', 'r') as file:
#     # 파일의 각 줄을 읽어와 리스트에 저장
#     y_preds = [int(line.strip()) for line in file]

In [ ]:
print(len(y_preds))

In [ ]:
len(y_true)

## 평가하기

In [ ]:
from inference.evaluate import print_confusion_matrix, evaluate_classifier
evaluate_classifier(y_true, y_preds)

In [ ]:
print_confusion_matrix(y_true, y_preds)